### Latex macros:
$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bo}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\e}{\frac{1}{\sigma^2_e}}
\newcommand{\f}{\frac{1}{\sigma^2_{\alpha}}}
$$

In [1]:
macro javascript_str(s) display("text/javascript", s); end
javascript"""
    MathJax.Hub.Config({
      TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
    MathJax.Hub.Queue( 
        ["resetEquationNumbers",MathJax.InputJax.TeX], 
        ["PreProcess",MathJax.Hub], 
        ["Reprocess",MathJax.Hub] 
    );
"""

# Iterative methods for solving linear systems

Consider the system of consistent linear equations: 

$$
\mathbf{Ax} = \mathbf{b},
$$
* Three iterative methods that we will use for solving the linear systems are the Jacobi method, the Gauss-Seidel Methods, and the Preconditioned Conjugate Gradient (PCCG) method. 
* These methods can be used to solve normal equations shown in the previous section and Mixed Model Equations (MME) we will covered later. 
* Consider MME, the left-hand-side (LHS) of the MME is represented by $\mathbf{A}$ and the right-hand-side (RHS) by $\mathbf{b}$. 
* The LHS of the MME is often too large to store in memory as a “fully-stored” matrix. However, $\mathbf{A}$ is often very sparse. Thus, it is may be possible to store only the non-zero elements of $\mathbf{A}$ and compute $\mathbf{Ax_n}$, using sparse matrix methods.

### The Jacobi method

In the Jacobi method, the solution at iteration $t+1$ can be written as: 
\begin{equation}
  \label{eq:jacobi}
\mathbf{x}^{t+1} = \mathbf{D}^{−1}(\mathbf{b} − \mathbf{A}\mathbf{x}^t) + \mathbf{x}^t,
\end{equation}
where $\mathbf{D}$ is the diagonal component of $\mathbf{A}$.

Convergence can often be improved by modifying (\ref{eq:jacobi}) as:
\begin{equation}
\mathbf{x}^{t+1*} = \alpha\mathbf{x}^{t+1} + (1 − \alpha)\mathbf{x}^{t*}   \label{eq:jacobi_modified}
\end{equation}
for $0 < \alpha < 1$.

Straightforward application of the Jacobi method to solve the normal equations or mixed model equations
would require first computing the left-hand-side (LHS) and right-hand-side (RHS) of the normal equations or mixed model equations, and then using (\ref{eq:jacobi_modified}) until convergence.

### Exercise

1. Consider the left-hand-side (LHS) and right-hand-side (RHS) of the normal equations given below. Take $\mathbf{x}^{0}$ as a vector of zeros, compute $\mathbf{x}^{1}$ using equation (1). How can you check the convergence of this iterative method?
2. Use equation (1) again to compute $\mathbf{x}^{2}$ from $\mathbf{x}^{1}$ computed previously and check convergence.
3. Write a loop to compute $\mathbf{x}^{100}$. How many iterations are required to reach convergence?

#### Data

In [2]:
using LinearAlgebra, Distributions, DataFrames

In [3]:
data = DataFrame(x=[1,1,2,2,2,2,3,3,4,1],y=[1.1,1.2,1.9,1.2,2.0,1.7,1.0,1.7,1.1,1.7])
n = size(data,1)
p = length(unique(data[:x]))
X = zeros(n,p);
for i = 1:n
    j = data[:x][i]
    X[i,j] = 1.0
end
X = [ones(n) X];
y = data[:y];
lhs = X'X;
rhs = X'y;

In [4]:
lhs

5×5 Array{Float64,2}:
 10.0  3.0  4.0  2.0  1.0
  3.0  3.0  0.0  0.0  0.0
  4.0  0.0  4.0  0.0  0.0
  2.0  0.0  0.0  2.0  0.0
  1.0  0.0  0.0  0.0  1.0

In [5]:
D = Diagonal(lhs)
A = lhs
b = rhs
#approach 1
println("approach 1")
x = ones(5)
for i = 1:4
    x=inv(D)*(b-A*x)+x
    println(x)
end

#approach 2
println("approach 2")
x = ones(5)
x=inv(D)*(b-A*x)+x
println(x)
x=inv(D)*(b-A*x)+x
println(x)
x=inv(D)*(b-A*x)+x
println(x)
x=inv(D)*(b-A*x)+x
println(x)

approach 1
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
approach 2
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]


#### solution

In [6]:
function Jacobi(A,x,b,p=0.7;tolerance=0.000001,printout_frequency=10,maxiter=1000) #optional arguments keyword arguments
    n       = size(A,1)        #number of linear equations
    D       = diag(A)
    error   = b - A*x
    diff    = sum(error.^2)/n
    
    iter    = 0
    while (diff > tolerance) && (iter < maxiter)
        iter   += 1
        error   = b - A*x
        x_temp  = error./D + x
        x       = p*x_temp + (1-p)*x
        diff    = sum(error.^2)/n
        
        if iter%printout_frequency == 0
            println(iter," ",diff)
        end
    end
    return x
end

Jacobi (generic function with 2 methods)

In [7]:
starting_value = zeros(size(lhs,2))
sol=Jacobi(lhs,starting_value,rhs,0.7)
[X*sol X*(lhs\rhs)]

10 3.8029144498142352e-6


10×2 Array{Float64,2}:
 1.33339  1.33333
 1.33339  1.33333
 1.70006  1.7    
 1.70006  1.7    
 1.70006  1.7    
 1.70006  1.7    
 1.35006  1.35   
 1.35006  1.35   
 1.10006  1.1    
 1.33339  1.33333

### The Gauss–Seidel method

In the Gauss–Seidel method, the solution at iteration $t + 1$ can be written as: 

\begin{equation}
\mathbf{x}^{t+1} = \mathbf{L}^{−1}(\mathbf{b} − \mathbf{U}\mathbf{x}^t), \label{gs} 
\end{equation}

where $\mathbf{L}$ and $\mathbf{U}$ are lower triangular component and strictly upper triangular component of $\mathbf{A}$.

This can also be computed as:
$$
x^{t+1}_{i} = \frac{1}{A_{ii}}(b_i − \mathbf{A}_{i,1:(i-1)}\mathbf{x}^{t+1}_{1:(i-1)}-
\mathbf{A}_{i,(i+1):n}\mathbf{x}^{t}_{(i+1):n}),
$$
where $n$ is the number of linear equations.

### Exercise

1. Take $\mathbf{x}^{0}$ as a vector of zeros, compute $\mathbf{x}^{1}$ using equation (3).
2. Modify the Jacobi function for Gauss-Seidel iteration.

In [8]:
function GaussSeidel(A,x,b;tolerance=0.000001,printout_frequency=10,maxiter=1000)
    n = size(A,1)
    for i = 1:n
        x[i] = (b[i] - A[:,i]'x)/A[i,i] + x[i]
    end
    error = b - A*x
    diff  = sum(error.^2)/n
    
    iter  = 0
    while (diff > tolerance) & (iter < maxiter)
        iter += 1
        for i = 1:n
            x[i] = (b[i] - A[:,i]'x)/A[i,i] + x[i]
        end
        
        error = b - A*x
        diff  = sum(error.^2)/n
        if iter%printout_frequency == 0
            println(iter," ",diff)
        end
    end
    return x
end

GaussSeidel (generic function with 1 method)

In [9]:
starting_value = zeros(size(lhs,2))
sol = GaussSeidel(lhs,starting_value,rhs)
[X*sol X*(lhs\rhs)]

10×2 Array{Float64,2}:
 1.33333  1.33333
 1.33333  1.33333
 1.7      1.7    
 1.7      1.7    
 1.7      1.7    
 1.7      1.7    
 1.35     1.35   
 1.35     1.35   
 1.1      1.1    
 1.33333  1.33333

### [Conjugate Gradient Method](3.4.ConjugateGradient.ipynb)